In [1]:
# !pip install gym
# !pip install pygame
# ! pip install matplotlib


# https://www.gymlibrary.ml/environments/classic_control/cart_pole/?highlight=cart+pole

In [2]:
import gym
import numpy as np 
import matplotlib.pyplot as plt
import time

c:\Users\rossi\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\monitoring\video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


In [3]:
env = gym.make('CartPole-v1')
print(env.observation_space.low,"\n",env.observation_space.high)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38] 
 [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [4]:
action_space = 2 # number of possible actions
bin_size = 4
infinity_linspace_limit = 4

overe0 = float(4)
overe1 = float(4)/2**1
overe2 = float(4)/2**2
overe3 = float(4)/2**3
overe4 = float(4)/2**4
overe5 = float(4)/2**5
overe6 = float(4)/2**6

lin_space = [-overe0, -overe1, -overe2, -overe3, -overe4, -overe5, -overe6, 0, overe6, overe5, overe4, overe3, overe2, overe1, overe0]

def Qtable(state_space,action_space,bin_size = 15):

    bins = [np.linspace(-2.4,2.4,bin_size),
            lin_space,
            np.linspace(-0.2095,0.2095,bin_size),
            lin_space]
    # bins = [np.linspace(-2.4,2.4,bin_size),
    #         np.linspace(-infinity_linspace_limit,infinity_linspace_limit,bin_size),
    #         np.linspace(-0.2095,0.2095,bin_size),
    #         np.linspace(-infinity_linspace_limit,infinity_linspace_limit,bin_size)]
    
    q_table = np.random.uniform(low=-1,high=1,size=([len(bins[0]), len(bins[1]), len(bins[2]), len(bins[3])] + [action_space]))

    return q_table, bins

def Discrete(state, bins):
    index = []
    for i in range(len(state)): index.append(np.digitize(state[i],bins[i]) - 1)
    return tuple(index)

In [5]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q[state])
   
    return action

In [6]:
def plotLearning(data, ep):
        plt.plot(ep, data['max'], label = 'Max')
        plt.plot(ep, data['avg'], label = 'Avg')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.legend(loc = "upper left")

In [13]:
    
def Q_learning(q_table, bins, episodes = 5000, gamma = 0.95, lr = 0.1, timestep = 5000, epsilon = 0.2):
    rewards = 0
    solved = False 
    runs = [0]
    data = {'max' : [0], 'avg' : [0]}
    ep = [i for i in range(0,episodes + 1,timestep)] 

    
    for episode in range(1,episodes+1):
        current_state = Discrete(env.reset(),bins) # initial observation
        score = 0
        done = False
        
        while not done:
            ep_start = time.time()
            # if episode%timestep == 0:
            #     env.render()
                
            action = epsilon_greedy_policy(current_state, q_table, epsilon)
            
            obs, reward, done, info = env.step(action)
            next_state = Discrete(obs,bins)

            score += reward
            
            # if not done:
            max_future_q = np.max(q_table[next_state])
            current_q = q_table[current_state+(action,)]
            new_q = (1-lr)*current_q + lr*(reward + gamma*max_future_q)
            q_table[current_state+(action,)] = new_q

            current_state = next_state
            
        # End of the loop update
        else:
            env.reset()
            rewards += score
            runs.append(score)

            if score > 500 and solved == False: # considered as a solved:
                solved = True
                print('Solved in episode : {} in time {}'.format(episode, (time.time()-ep_start)))
        
        # Timestep value update
        if episode%timestep == 0:
            print('Episode : {} | Avg. Rewards -> {} | Max reward : {} | Time : {}'.format(episode,rewards/timestep, max(runs), time.time() - ep_start))
            data['max'].append(max(runs))
            data['avg'].append(rewards/timestep)
            if rewards/timestep >= 475: 
                print('Solved in episode : {}'.format(episode))
            rewards, runs= 0, [0]
            print(len(data['max'])) 

          
    if len(ep) == len(data['max']):
        plt.plot(ep, data['max'], label = 'Max')
        plt.plot(ep, data['avg'], label = 'Avg')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.legend(loc = "upper left")
        
    env.close()

In [14]:
# TRANING
q_table, bins = Qtable(len(env.observation_space.low), env.action_space.n)
# Q_learning(q_table, bins, lr = 0.12, gamma = 0.995, episodes = 2*10**5, timestep = 1000, epsilon = 0.05)
Q_learning(q_table, bins, lr = 0.12, gamma = 0.995, episodes = 80000, timestep = 1000, epsilon = 0.06)

Episode : 1000 | Avg. Rewards -> 30.717 | Max reward : 140.0 | Time : 0.0009589195251464844
2
Episode : 2000 | Avg. Rewards -> 28.718 | Max reward : 147.0 | Time : 0.0
2
Episode : 3000 | Avg. Rewards -> 39.484 | Max reward : 161.0 | Time : 0.0
2
Episode : 4000 | Avg. Rewards -> 48.675 | Max reward : 244.0 | Time : 0.0
2
Episode : 5000 | Avg. Rewards -> 55.81 | Max reward : 215.0 | Time : 0.0
2
Episode : 6000 | Avg. Rewards -> 65.685 | Max reward : 327.0 | Time : 0.0
2


KeyboardInterrupt: 